# Tutorial 03: Running RLlib Experiments

This tutorial walks you through the process of running traffic simulations in Flow with trainable RLlib-powered agents. Autonomous agents will learn to maximize a certain reward over the rollouts, using the [**RLlib**](https://ray.readthedocs.io/en/latest/rllib.html) library ([citation](https://arxiv.org/abs/1712.09381)) ([installation instructions](https://flow.readthedocs.io/en/latest/flow_setup.html#optional-install-ray-rllib)). Simulations of this form will depict the propensity of RL agents to influence the traffic of a human fleet in order to make the whole fleet more efficient (for some given metrics). 

In this tutorial, we simulate an initially perturbed single lane ring road, where we introduce a single autonomous vehicle. We witness that, after some training, that the autonomous vehicle learns to dissipate the formation and propagation of "phantom jams" which form when only human driver dynamics are involved.

## 1. Components of a Simulation
All simulations, both in the presence and absence of RL, require two components: a *network*, and an *environment*. Networks describe the features of the transportation network used in simulation. This includes the positions and properties of nodes and edges constituting the lanes and junctions, as well as properties of the vehicles, traffic lights, inflows, etc... in the network. Environments, on the other hand, initialize, reset, and advance simulations, and act as the primary interface between the reinforcement learning algorithm and the network. Moreover, custom environments may be used to modify the dynamical features of an network. Finally, in the RL case, it is in the *environment* that the state/action spaces and the reward function are defined. 

## 2. Setting up a Network
Flow contains a plethora of pre-designed networks used to replicate highways, intersections, and merges in both closed and open settings. All these networks are located in flow/networks. For this tutorial, which involves a single lane ring road, we will use the network `RingNetwork`.

### 2.1 Setting up Network Parameters

The network mentioned at the start of this section, as well as all other networks in Flow, are parameterized by the following arguments: 
* name
* vehicles
* net_params
* initial_config

These parameters are explained in detail in `tutorial01_sumo.ipynb`. Moreover, all parameters excluding vehicles (covered in section 2.2) do not change from the previous tutorial. Accordingly, we specify them nearly as we have before, and leave further explanations of the parameters to `tutorial01_sumo.ipynb`.

We begin by choosing the network the experiment will be trained on. We use one of Flow's builtin networks, located in `flow.networks`. A list of all available networks can be found by running the script below.

In [1]:
import flow.networks as networks

# print(networks.__all__)

In this tutorial, we choose to use the ring road network. The network class is then:

In [2]:
from flow.networks import RingNetwork

# ring road network class
network_name = RingNetwork

One key difference between SUMO and RLlib experiments is that, in RLlib experiments, the network classes do not need to be defined; instead users should simply name the network class they wish to use. Later on, an environment setup module will import the correct network class based on the provided names.

In [3]:
# input parameter classes to the network class
from flow.core.params import NetParams, InitialConfig

# name of the network
name = "c_mpg+plus"

# network-specific parameters
from flow.networks.ring import ADDITIONAL_NET_PARAMS
net_params = NetParams(additional_params=ADDITIONAL_NET_PARAMS)

# initial configuration to vehicles
initial_config = InitialConfig(spacing="uniform", perturbation=1)

### 2.2 Adding Trainable Autonomous Vehicles
The `Vehicles` class stores state information on all vehicles in the network. This class is used to identify the dynamical features of a vehicle and whether it is controlled by a reinforcement learning agent. Morover, information pertaining to the observations and reward function can be collected from various `get` methods within this class.

The dynamics of vehicles in the `Vehicles` class can either be depicted by sumo or by the dynamical methods located in flow/controllers. For human-driven vehicles, we use the IDM model for acceleration behavior, with exogenous gaussian acceleration noise with std 0.2 m/s2 to induce perturbations that produce stop-and-go behavior. In addition, we use the `ContinousRouter` routing controller so that the vehicles may maintain their routes closed networks.

As we have done in `tutorial01_sumo.ipynb`, human-driven vehicles are defined in the `VehicleParams` class as follows:

In [4]:
# vehicles class
from flow.core.params import VehicleParams

# vehicles dynamics models
from flow.controllers import IDMController, ContinuousRouter

vehicles = VehicleParams()
#vehicles.add("human",
#             acceleration_controller=(IDMController, {}),
#             routing_controller=(ContinuousRouter, {}),
#             num_vehicles=10)

The above addition to the `Vehicles` class only accounts for 21 of the 22 vehicles that are placed in the network. We now add an additional trainable autuonomous vehicle whose actions are dictated by an RL agent. This is done by specifying an `RLController` as the acceleraton controller to the vehicle. 

In [5]:
from flow.controllers import RLController

Note that this controller serves primarirly as a placeholder that marks the vehicle as a component of the RL agent, meaning that lane changing and routing actions can also be specified by the RL agent to this vehicle.

We finally add the vehicle as follows, while again using the `ContinuousRouter` to perpetually maintain the vehicle within the network.

In [6]:
# from flow.energy_models.toyota_energy import TacomaEnergy
# vehicles.add(veh_id="rl",
#              acceleration_controller=(RLController, {}),
#              routing_controller=(ContinuousRouter, {}),
#              initial_speed =20,
#              energy_model = TacomaEnergy,
#              num_vehicles=1)


vehicles.add(veh_id="rl",
             acceleration_controller=(RLController, {}),
             routing_controller=(ContinuousRouter, {}),
             initial_speed =0,
             num_vehicles=1)

## 3. Setting up an Environment

Several environments in Flow exist to train RL agents of different forms (e.g. autonomous vehicles, traffic lights) to perform a variety of different tasks. The use of an environment allows us to view the cumulative reward simulation rollouts receive, along with to specify the state/action spaces.

Sumo envrionments in Flow are parametrized by three components:
* `SumoParams`
* `EnvParams`
* `Network`

### 3.1 SumoParams
`SumoParams` specifies simulation-specific variables. These variables include the length of any simulation step and whether to render the GUI when running the experiment. For this example, we consider a simulation step length of 0.1s and deactivate the GUI. 

**Note** For training purposes, it is highly recommanded to deactivate the GUI in order to avoid global slow down. In such case, one just needs to specify the following: `render=False`

In [7]:
from flow.core.params import SumoParams

sim_params = SumoParams(sim_step=0.1, render=False)

### 3.2 EnvParams

`EnvParams` specifies environment and experiment-specific parameters that either affect the training process or the dynamics of various components within the network. For the environment `WaveAttenuationPOEnv`, these parameters are used to dictate bounds on the accelerations of the autonomous vehicles, as well as the range of ring lengths (and accordingly network densities) the agent is trained on.

Finally, it is important to specify here the *horizon* of the experiment, which is the duration of one episode (during which the RL-agent acquire data). 

In [8]:
from flow.core.params import EnvParams

# Define horizon as a variable to ensure consistent use across notebook
HORIZON=2500

env_params = EnvParams(
    # length of one rollout
    horizon=HORIZON,

    additional_params={
        # maximum acceleration of autonomous vehicles
        "max_accel": 4,
        # maximum deceleration of autonomous vehicles
        "max_decel": -4,
        # bounds on the ranges of ring road lengths the autonomous vehicle 
        # is trained on
        "ring_length": [220, 270],
    },
)

### 3.3 Initializing a Gym Environment

Now, we have to specify our Gym Environment and the algorithm that our RL agents will use. Similar to the network, we choose to use on of Flow's builtin environments, a list of which is provided by the script below.

In [9]:
import flow.envs as flowenvs

print(flowenvs.__all__)

['Env', 'AccelEnv', 'LaneChangeAccelEnv', 'LaneChangeAccelPOEnv', 'TrafficLightGridTestEnv', 'MergePOEnv', 'BottleneckEnv', 'BottleneckAccelEnv', 'WaveAttenuationEnv', 'WaveAttenuationPOEnv', 'EnergyOptEnv', 'EnergyOptSPDEnv', 'TrafficLightGridEnv', 'TrafficLightGridPOEnv', 'TrafficLightGridBenchmarkEnv', 'BottleneckDesiredVelocityEnv', 'TestEnv', 'BayBridgeEnv', 'SingleStraightRoad', 'BottleNeckAccelEnv', 'DesiredVelocityEnv', 'PO_TrafficLightGridEnv', 'GreenWaveTestEnv']


We will use the environment "WaveAttenuationPOEnv", which is used to train autonomous vehicles to attenuate the formation and propagation of waves in a partially observable variable density ring road. To create the Gym Environment, the only necessary parameters are the environment name plus the previously defined variables. These are defined as follows:

In [10]:
from flow.envs import EnergyOptSPDEnv

env_name = EnergyOptSPDEnv

In [11]:
# from flow.envs import WaveAttenuationPOEnv

# env_name = WaveAttenuationPOEnv

### 3.4 Setting up Flow Parameters

RLlib experiments both generate a `params.json` file for each experiment run. For RLlib experiments, the parameters defining the Flow network and environment must be stored as well. As such, in this section we define the dictionary `flow_params`, which contains the variables required by the utility function `make_create_env`. `make_create_env` is a higher-order function which returns a function `create_env` that initializes a Gym environment corresponding to the Flow network specified.

In [12]:
# Creating flow_params. Make sure the dictionary keys are as specified. 
flow_params = dict(
    # name of the experiment
    exp_tag=name,
    # name of the flow environment the experiment is running on
    env_name=env_name,
    # name of the network class the experiment uses
    network=network_name,
    # simulator that is used by the experiment
    simulator='traci',
    # simulation-related parameters
    sim=sim_params,
    # environment related parameters (see flow.core.params.EnvParams)
    env=env_params,
    # network-related parameters (see flow.core.params.NetParams and
    # the network's documentation or ADDITIONAL_NET_PARAMS component)
    net=net_params,
    # vehicles to be placed in the network at the start of a rollout 
    # (see flow.core.vehicles.Vehicles)
    veh=vehicles,
    # (optional) parameters affecting the positioning of vehicles upon 
    # initialization/reset (see flow.core.params.InitialConfig)
    initial=initial_config
)

## 4 Running RL experiments in Ray

### 4.1 Import 

First, we must import modules required to run experiments in Ray. The `json` package is required to store the Flow experiment parameters in the `params.json` file, as is `FlowParamsEncoder`. Ray-related imports are required: the PPO algorithm agent, `ray.tune`'s experiment runner, and environment helper methods `register_env` and `make_create_env`.

In [13]:
import json

import ray
try:
    from ray.rllib.agents.agent import get_agent_class
except ImportError:
    from ray.rllib.agents.registry import get_agent_class
# from ray.rllib.agents.agent import get_agent_class
#from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env

from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder

Instructions for updating:
non-resource variables are not supported in the long term


### 4.2 Initializing Ray
Here, we initialize Ray and experiment-based constant variables specifying parallelism in the experiment as well as experiment batch size in terms of number of rollouts.

In [14]:
# number of parallel workers
N_CPUS = 6
# number of rollouts per training iteration
N_ROLLOUTS = 1
#ray.shutdown()
ray.init(num_cpus=N_CPUS)

2020-07-30 18:03:30,973	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2020-07-30_18-03-30_972842_25964/logs.
2020-07-30 18:03:31,092	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:55213 to respond...
2020-07-30 18:03:31,235	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:58697 to respond...
2020-07-30 18:03:31,241	INFO services.py:809 -- Starting Redis shard with 3.3 GB max memory.
2020-07-30 18:03:31,298	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2020-07-30_18-03-30_972842_25964/logs.
2020-07-30 18:03:31,300	WARNING services.py:1330 -- WARNING: The default object store size of 4.96 GB will use more than 50% of the available memory on this node (7.52 GB). Consider setting the object store memory manually to a smaller size to avoid memory contention with other applications.
2020-07-30 18:03:31,301	INFO services.py:1475 -- Starting the Plasma object store with 4.96 GB memory us

{'node_ip_address': '192.168.100.38',
 'redis_address': '192.168.100.38:55213',
 'object_store_address': '/tmp/ray/session_2020-07-30_18-03-30_972842_25964/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-07-30_18-03-30_972842_25964/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2020-07-30_18-03-30_972842_25964'}

### 4.3 Configuration and Setup
Here, we copy and modify the default configuration for the [PPO algorithm](https://arxiv.org/abs/1707.06347). The agent has the number of parallel workers specified, a batch size corresponding to `N_ROLLOUTS` rollouts (each of which has length `HORIZON` steps), a discount rate $\gamma$ of 0.999, two hidden layers of size 16, uses Generalized Advantage Estimation, $\lambda$ of 0.97, and other parameters as set below.

Once `config` contains the desired parameters, a JSON string corresponding to the `flow_params` specified in section 3 is generated. The `FlowParamsEncoder` maps objects to string representations so that the experiment can be reproduced later. That string representation is stored within the `env_config` section of the `config` dictionary. Later, `config` is written out to the file `params.json`. 

Next, we call `make_create_env` and pass in the `flow_params` to return a function we can use to register our Flow environment with Gym. 

In [15]:
# The algorithm or model to train. This may refer to "
#      "the name of a built-on algorithm (e.g. RLLib's DQN "
#      "or PPO), or a user-defined trainable function or "
#      "class registered in the tune registry.")
alg_run = "PPO"

agent_cls = get_agent_class(alg_run)
config = agent_cls._default_config.copy()
config["num_workers"] = N_CPUS - 1  # number of parallel workers
config["train_batch_size"] = HORIZON * N_ROLLOUTS  # batch size
config["gamma"] = 0.99999  # discount rate
config["model"].update({"fcnet_hiddens": [16, 16]})  # size of hidden layers in network
config["use_gae"] = True  # using generalized advantage estimation
config["lambda"] = 0.97  
config["sgd_minibatch_size"] = min(16 * 1024, config["train_batch_size"])  # stochastic gradient descent
config["kl_target"] = 0.02  # target KL divergence
config["num_sgd_iter"] = 10  # number of SGD iterations
config["horizon"] = HORIZON  # rollout horizon

# save the flow params for replay
flow_json = json.dumps(flow_params, cls=FlowParamsEncoder, sort_keys=True,
                       indent=4)  # generating a string version of flow_params
config['env_config']['flow_params'] = flow_json  # adding the flow_params to config dict
config['env_config']['run'] = alg_run

# Call the utility function make_create_env to be able to 
# register the Flow env for this experiment
create_env, gym_name = make_create_env(params=flow_params, version=0)

# Register as rllib env with Gym
register_env(gym_name, create_env)

### 4.4 Running Experiments

Here, we use the `run_experiments` function from `ray.tune`. The function takes a dictionary with one key, a name corresponding to the experiment, and one value, itself a dictionary containing parameters for training.

In [16]:
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,
        "env": gym_name,
        "config": {
            **config
        },
        "checkpoint_freq": 20,  # number of iterations between checkpoints
        "checkpoint_at_end": True,  # generate a checkpoint at the end
        "max_failures": 999,
        "stop": {  # stopping conditions
            "training_iteration": 1500,  # number of iterations to stop after
        },
    },
})

2020-07-30 18:03:31,610	INFO trial_runner.py:176 -- Starting a new experiment.
2020-07-30 18:03:31,704	WARNING signature.py:108 -- The function with_updates has a **kwargs argument, which is currently not supported.
2020-07-30 18:03:31,726	WARNING logger.py:227 -- Could not instantiate <class 'ray.tune.logger.TFLogger'> - skipping.
2020-07-30 18:03:31,736	ERROR log_sync.py:34 -- Log sync requires cluster to be setup with `ray up`.
2020-07-30 18:03:31,814	WARNING util.py:145 -- The `start_trial` operation took 0.11550617218017578 seconds to complete, which may be a performance bottleneck.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/6 CPUs, 0/0 GPUs
Memory usage on this node: 9.1/16.5 GB

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 6/6 CPUs, 0/0 GPUs
Memory usage on this node: 9.1/16.5 GB
Result logdir: /home/solom/ray_results/c_mpg+plus
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - PPO_EnergyOptSPDEnv-v0_0:	RUNNING

(pid=26009) WARNING:tensorflow:From /home/solom/anaconda3/lib/python3.7/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=26009) Instructions for updating:
(pid=26009) non-resource variables are not supported in the long term
(pid=26009) 2020-07-30 18:03:34,326	WARNING ppo.py:143 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).
(pid=26009) 2020-07-30 18:03:35,444	INFO rollout_worker.py:319 -- Creating poli

(pid=26007) WARNING:tensorflow:From /home/solom/anaconda3/lib/python3.7/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=26007) Instructions for updating:
(pid=26007) non-resource variables are not supported in the long term
(pid=26009) 2020-07-30 18:03:39,560	WARNING util.py:47 -- Install gputil for GPU system monitoring.
(pid=26009) WARNING:tensorflow:From /home/solom/anaconda3/lib/python3.7/site-packages/ray/rllib/policy/tf_policy.py:570: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
(pid=26009) Instructions for updating:
(pid=26009) Prefer Variable.assign which has equivalent behavior in 2.X.
(pid=26009) WARNING:tensorflow:From /home/solom/anaconda3/lib/python3.7/site-packages/ray/rllib/policy/tf_policy.py:570: Variable.load (from tensorflow.python.ops.variables) is deprecated and will

(pid=26008) WARNING:tensorflow:From /home/solom/anaconda3/lib/python3.7/site-packages/ray/rllib/models/tf/tf_action_dist.py:138: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
(pid=26008) Instructions for updating:
(pid=26008) Use `tf.cast` instead.
(pid=26008) WARNING:tensorflow:From /home/solom/anaconda3/lib/python3.7/site-packages/ray/rllib/models/tf/tf_action_dist.py:138: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
(pid=26008) Instructions for updating:
(pid=26008) Use `tf.cast` instead.
(pid=26004) WARNING:tensorflow:From /home/solom/anaconda3/lib/python3.7/site-packages/ray/rllib/models/tf/tf_action_dist.py:138: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
(pid=26004) Instructions for updating:
(pid=26004) Use `tf.cast` instead.
(pid=26004) WARNING:tensorflow:From /home/solom/anaconda3/lib/python3.7/site-packages

(pid=26008) WARNING:tensorflow:From /home/solom/anaconda3/lib/python3.7/site-packages/ray/rllib/policy/tf_policy.py:570: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
(pid=26008) Instructions for updating:
(pid=26008) Prefer Variable.assign which has equivalent behavior in 2.X.
(pid=26008) WARNING:tensorflow:From /home/solom/anaconda3/lib/python3.7/site-packages/ray/rllib/policy/tf_policy.py:570: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
(pid=26008) Instructions for updating:
(pid=26008) Prefer Variable.assign which has equivalent behavior in 2.X.
(pid=26008) 
(pid=26008) -----------------------
(pid=26008) ring length: 236
(pid=26008) -----------------------
(pid=26004) 2020-07-30 18:03:42,986	INFO sampler.py:304 -- Raw obs from env: { 0: { 'agent0': np.ndarray((3,), dtype=float64, min=0.0, max=0.0, mean=0.0)}}
(pid=26004) 2020-07-30 18:03:42,986	INFO sampler

...

(pid=26006) 
(pid=26006) -----------------------
(pid=26006) ring length: 242
(pid=26006) -----------------------
(pid=26007) 
(pid=26007) -----------------------
(pid=26007) ring length: 244
(pid=26007) -----------------------
(pid=26005) 
(pid=26005) -----------------------
(pid=26005) ring length: 268
(pid=26005) -----------------------
Result for PPO_EnergyOptSPDEnv-v0_0:
  custom_metrics: {}
  date: 2020-07-30_19-17-55
  done: false
  episode_len_mean: 2500.0
  episode_reward_max: 1035.1014877303473
  episode_reward_mean: 473.8131959305023
  episode_reward_min: 140.2443830298491
  episodes_this_iter: 3
  episodes_total: 1204
  experiment_id: 0227ad3d84eb4f9da7b6d7d230fb1c49
  hostname: solom-XPS-13-9380
  info:
    grad_time_ms: 39.902
    learner:
      default_policy:
        cur_kl_coeff: 0.0
        cur_lr: 4.999999873689376e-05
        entropy: 1.4413940906524658
        entropy_coeff: 0.0
        kl: 1.2979506891497294e-07
        policy_loss: -0.0017853431636467576
        

(pid=26007) 
(pid=26007) -----------------------
(pid=26007) ring length: 220
(pid=26007) -----------------------
(pid=26006) 
(pid=26006) -----------------------
(pid=26006) ring length: 257
(pid=26006) -----------------------
Result for PPO_EnergyOptSPDEnv-v0_0:
  custom_metrics: {}
  date: 2020-07-30_19-18-23
  done: false
  episode_len_mean: 2500.0
  episode_reward_max: 1035.1014877303473
  episode_reward_mean: 469.54169434684974
  episode_reward_min: 140.2443830298491
  episodes_this_iter: 0
  episodes_total: 1209
  experiment_id: 0227ad3d84eb4f9da7b6d7d230fb1c49
  hostname: solom-XPS-13-9380
  info:
    grad_time_ms: 38.579
    learner:
      default_policy:
        cur_kl_coeff: 0.0
        cur_lr: 4.999999873689376e-05
        entropy: 1.443091630935669
        entropy_coeff: 0.0
        kl: 5.369901714402658e-07
        policy_loss: 0.0017986358143389225
        total_loss: 36.84474182128906
        vf_explained_var: 0.0030093789100646973
        vf_loss: 36.84294509887695
   

(pid=26004) 
(pid=26004) -----------------------
(pid=26004) ring length: 255
(pid=26004) -----------------------
(pid=26005) 
(pid=26005) -----------------------
(pid=26005) ring length: 227
(pid=26005) -----------------------
Result for PPO_EnergyOptSPDEnv-v0_0:
  custom_metrics: {}
  date: 2020-07-30_19-18-57
  done: false
  episode_len_mean: 2500.0
  episode_reward_max: 1035.1014877303473
  episode_reward_mean: 474.64640647136594
  episode_reward_min: 140.2443830298491
  episodes_this_iter: 1
  episodes_total: 1216
  experiment_id: 0227ad3d84eb4f9da7b6d7d230fb1c49
  hostname: solom-XPS-13-9380
  info:
    grad_time_ms: 47.66
    learner:
      default_policy:
        cur_kl_coeff: 0.0
        cur_lr: 4.999999873689376e-05
        entropy: 1.4475589990615845
        entropy_coeff: 0.0
        kl: 5.376339231588645e-07
        policy_loss: 0.008351102471351624
        total_loss: 32.87419509887695
        vf_explained_var: 0.002024412155151367
        vf_loss: 32.8658561706543
    lo

2020-07-30 19:19:03,582	ERROR import_thread.py:89 -- ImportThread: Connection closed by server.
2020-07-30 19:19:03,584	ERROR worker.py:1716 -- listen_error_messages_raylet: Connection closed by server.
2020-07-30 19:19:03,585	ERROR worker.py:1616 -- print_logs: Connection closed by server.


KeyboardInterrupt: 

### 4.5 Visualizing the results

The simulation results are saved within the `ray_results/training_example` directory (we defined `training_example` at the start of this tutorial). The `ray_results` folder is by default located at your root `~/ray_results`. 

You can run `tensorboard --logdir=~/ray_results/training_example` (install it with `pip install tensorboard`) to visualize the different data outputted by your simulation.

For more instructions about visualizing, please see `tutorial05_visualize.ipynb`. 

### 4.6 Restart from a checkpoint / Transfer learning

If you wish to do transfer learning, or to resume a previous training, you will need to start the simulation from a previous checkpoint. To do that, you can add a `restore` parameter in the `run_experiments` argument, as follows:

```python
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,
        "env": gym_name,
        "config": {
            **config
        },
        "restore": "/ray_results/experiment/dir/checkpoint_50/checkpoint-50"
        "checkpoint_freq": 1,
        "checkpoint_at_end": True,
        "max_failures": 999,
        "stop": {
            "training_iteration": 1,
        },
    },
})
```

The `"restore"` path should be such that the `[restore]/.tune_metadata` file exists.

There is also a `"resume"` parameter that you can set to `True` if you just wish to continue the training from a previously saved checkpoint, in case you are still training on the same experiment. 

In [ ]:
# trials = run_experiments({
#     flow_params["exp_tag"]: {
#         "run": alg_run,
#         "env": gym_name,
#         "config": {
#             **config
#         },
#         "restore": "/ray_results/training_example13/PPO_EnergyOptPOEnv-v0_0_2020-07-23_13-30-07yze28sum/checkpoint_400/checkpoint-400", 
#         "checkpoint_freq": 20,
#         "checkpoint_at_end": True,
#         "max_failures": 999,
#         "stop": {
#             "training_iteration": 700,
#         },
#     },
# })

In [ ]:
from flow.core.vehicles import Vehicles